In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import os
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load processed data from disk
save_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/final_DataSet'

X = np.load(os.path.join(save_path, 'X_i3d_100.npy'), mmap_mode='r')
y = np.load(os.path.join(save_path, 'y_i3d_100.npy'))

In [ ]:
# Split data into training and testing sets first
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the shapes of the splits to verify
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (896, 64, 224, 224, 3)
X_val shape: (112, 64, 224, 224, 3)
X_test shape: (112, 64, 224, 224, 3)


In [5]:
# Load the top 100 glosses
top_100_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/DataSet/gloss_counts_top_100.csv'
df = pd.read_csv(top_100_path)
top_100_classes = df['Gloss'].tolist()  # List of the top 100 glosses

In [6]:
# Ensure TensorFlow is using GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid memory errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is being used!")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Using CPU instead.")

GPU is being used!


In [16]:
# Define the I3D model with adjustable hyperparameters
def build_i3d_model(input_shape, num_classes, learning_rate=0.001, dropout_rate=0.5, l2_lambda=0.001):
    model = models.Sequential()

    # 3D convolution layers with BatchNormalization
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same', input_shape=input_shape))
    #model.add(layers.BatchNormalization())  # Adding BatchNormalization after Conv3D
    model.add(layers.MaxPooling3D((2, 2, 2)))

    model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
    #model.add(layers.BatchNormalization())  # Adding BatchNormalization after Conv3D
    model.add(layers.MaxPooling3D((2, 2, 2)))

    model.add(layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
    #model.add(layers.BatchNormalization())  # Adding BatchNormalization after Conv3D
    model.add(layers.MaxPooling3D((2, 2, 2)))

    model.add(layers.Conv3D(512, (3, 3, 3), activation='relu', padding='same'))
    #model.add(layers.BatchNormalization())  # Adding BatchNormalization after Conv3D
    model.add(layers.MaxPooling3D((2, 2, 2)))

    # Flattening layer
    model.add(layers.Flatten())

    # Fully connected layers with regularization
    model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
    model.add(layers.Dropout(dropout_rate))

    # Output layer for multi-class classification
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model



In [17]:
def data_generator(X_data, y_data, batch_size=4):
    size = len(X_data)
    while True:  # Loop forever so the generator never stops
        for i in range(0, size, batch_size):
            batch_x = X_data[i:i + batch_size]
            batch_y = y_data[i:i + batch_size]
            yield batch_x, batch_y

# Initialize the generator
train_gen = data_generator(X_train, y_train, batch_size=4)
val_gen = data_generator(X_val, y_val, batch_size=4)

In [18]:
# Define input shape and number of classes
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4])  # (num_frames, height, width, channels)
num_classes = len(top_100_classes)

# Build the model
model = build_i3d_model(input_shape, num_classes)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Use the generators for model training
history = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    steps_per_epoch=len(X_train) // 8,
    validation_steps=len(X_val) // 8,
    callbacks=[early_stopping]
)

# Print final validation accuracy and loss after training
final_val_accuracy = history.history['val_accuracy'][-1]
final_val_loss = history.history['val_loss'][-1]

Epoch 1/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 28s 196ms/step - accuracy: 0.0058 - loss: 15.7836 - val_accuracy: 0.0000e+00 - val_loss: 6.8447
Epoch 2/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step - accuracy: 0.0121 - loss: 6.5327 - val_accuracy: 0.0179 - val_loss: 5.8439
Epoch 3/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.0162 - loss: 5.6980 - val_accuracy: 0.0000e+00 - val_loss: 5.3914
Epoch 4/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.0058 - loss: 5.3156 - val_accuracy: 0.0179 - val_loss: 5.1391
Epoch 5/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.0041 - loss: 5.0765 - val_accuracy: 0.0000e+00 - val_loss: 4.9842
Epoch 6/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.0184 - loss: 4.9419 - val_accuracy: 0.0179 - val_loss: 4.8830
Epoch 7/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.0065 - loss: 4.8419 - val_accuracy: 0.0000e+00 - val_loss: 4.8255
Epoch 8/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy:

In [21]:
# Print validation loss and accuracy after training
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]}")
print(f"Final validation loss: {history.history['val_loss'][-1]}")

# Save the model after training as a Keras model
model_save_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/i3d_model_trained.keras'
model.save(model_save_path)

print(f"Model saved to {model_save_path}")

Final validation accuracy: 0.01785714365541935
Final validation loss: 4.650790214538574
Model saved to /content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/i3d_model_trained.keras
